Scarpe Data and insert District Index

In [1]:
#Import necessary libraries
import os
from bs4 import BeautifulSoup # Import BeautifulSoup
import time # Import the time module
import pandas as pd
import requests


In [2]:
#Scrape data from the Real Estate Website
all_projects = []

for page in range(1, 9):
    url = f"https://cafeland.vn/bang-gia-du-an/0-1-0-0-nokey/page-{page}/" if page > 1 else "https://cafeland.vn/bang-gia-du-an/0-1-0-0-nokey/"
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(response.content, "html.parser")

    rows = soup.select("table#pricing-matrix tbody tr")[1:]  # skip header row

    for row in rows:
        cols = row.find_all("td")
        if len(cols) < 9:
            continue
        all_projects.append({
            "Mã số": cols[0].text.strip(),
            "Tên dự án": cols[1].text.strip(),
            "Link chi tiết": cols[1].find("a")["href"].strip() if cols[1].find("a") else "",
            "Loại dự án": cols[2].text.strip(),
            "Tình trạng": cols[3].text.strip(),
            "Diện tích (m2)": cols[4].text.strip(),
            "Giá (triệu đồng)": cols[5].text.strip(),
            "+/- %": cols[6].text.strip(),
            "Vị trí (map)": cols[7].find("a")["href"].strip() if cols[7].find("a") else "",
            "Liên hệ": cols[8].find("a")["href"].strip() if cols[8].find("a") else ""
        })



In [3]:
# Save to CSV
df = pd.DataFrame(all_projects)
df.to_csv("cafeland_projects_8pages.csv", index=False, encoding="utf-8-sig")
print("Saved to cafeland_projects_8pages.csv")

Saved to cafeland_projects_8pages.csv


In [4]:
df

,Mã số,Tên dự án,Link chi tiết,Loại dự án,Tình trạng,Diện tích (m2),Giá (triệu đồng),+/- %,Vị trí (map),Liên hệ
0,13354,Căn hộ chung cư Hacinco Nguyễn Xiển,https://cafeland.vn/du-an/hacinco-nguyen-xien-...,Căn hộ chung cư,,58 - 103,"1,102 - 1,957",,https://nhadat.cafeland.vn/map/tin-du-an/20.97...,/newsletter/3-4219/
1,13350,Căn hộ chung cư Green Tower,https://cafeland.vn/du-an/green-tower-du-an-nh...,Căn hộ chung cư,,56.8 - 82,"1,136 - 1,640",,https://nhadat.cafeland.vn/map/tin-du-an/20.99...,/newsletter/3-4208/
2,13332,Nhà Phố - Biệt Thự HCCI Hoàng Liệt,https://cafeland.vn/du-an/hcci-hoang-liet-du-a...,Nhà Phố - Biệt Thự,,128 - 100,"20,480 - 16,000",,https://nhadat.cafeland.vn/map/tin-du-an/20.96...,/newsletter/3-4152/
3,13329,Khu Đô Thị Palmy Biztown,https://cafeland.vn/du-an/palmy-biztown-du-an-...,Khu Đô Thị,,75 - 90,"5,025 - 6,030",,https://nhadat.cafeland.vn/map/tin-du-an/20.96...,/newsletter/3-4139/
4,13317,Căn hộ chung cư CT3 Nghĩa Đô,https://cafeland.vn/du-an/ct3-nghia-do-du-an-c...,Căn hộ chung cư,,43 - 69,"1,720 - 2,760",,https://nhadat.cafeland.vn/map/tin-du-an/21.05...,/newsletter/3-4122/
...,...,...,...,...,...,...,...,...,...,...
369,10566,Căn hộ chung cư Làng BIDV,https://cafeland.vn/du-an/lang-bidv-du-an-nong...,Căn hộ chung cư,,68 - 142,,,https://nhadat.cafeland.vn/map/tin-du-an/21.16...,/newsletter/3-322/
370,10558,Dolphin Plaza,https://cafeland.vn/du-an/khu-can-ho-dolphin-p...,Đang cập nhật,,130 - 198,,,https://nhadat.cafeland.vn/map/tin-du-an/21.03...,/newsletter/3-399/
371,10535,Richland Southern,https://cafeland.vn/du-an/khu-can-ho-cao-cap-r...,Đang cập nhật,,90 - 120,,,https://nhadat.cafeland.vn/map/tin-du-an/21.03...,/newsletter/3-287/
372,10041,Royal City,https://cafeland.vn/du-an/royal-city-uu-the-vu...,Đang cập nhật,,88 - 200,"3,256 - 7,400",,https://nhadat.cafeland.vn/map/tin-du-an/21.00...,/newsletter/3-271/


In [ ]:
#Extract Latitude and longtitude indexes
def extract_lat_long(url):
    try:
        parts = url.strip("/").split("/")
        lat = float(parts[-3])
        lon = float(parts[-2])
        return pd.Series([lat, lon])
    except Exception:
        return pd.Series([None, None])

df[['latitude', 'longitude']] = df['Vị trí (map)'].apply(extract_lat_long)

# Show a few results
print(df[['Tên dự án', 'latitude', 'longitude']].head())


                             Tên dự án   latitude   longitude
0  Căn hộ chung cư Hacinco Nguyễn Xiển  20.977536  105.814117
1          Căn hộ chung cư Green Tower  20.991173  105.760270
2   Nhà Phố - Biệt Thự HCCI Hoàng Liệt  20.966890  105.838003
3             Khu Đô Thị Palmy Biztown  20.967420  105.814353
4         Căn hộ chung cư CT3 Nghĩa Đô  21.050124  105.793117


In [ ]:
df.head()

In [ ]:
# Clean the area column before extracting numbers
df['Diện tích (m2)'] = df['Diện tích (m2)'].str.replace(',', '', regex=True)
# Split area and price into min/max
df[['area_min', 'area_max']] = df['Diện tích (m2)'].str.extract(r'([\d.]+)\s*-\s*([\d.]+)').astype(float)
# Clean the price column before extracting numbers
df['Giá (triệu đồng)'] = df['Giá (triệu đồng)'].str.replace(',', '', regex=True)

# Now extract correct min and max prices
df[['price_min', 'price_max']] = df['Giá (triệu đồng)'].str.extract(r'(\d+\.?\d*)\s*-\s*(\d+\.?\d*)').astype(float)

# Calculate average area and price
df['area_avg'] = (df['area_min'] + df['area_max']) / 2
df['price_avg'] = (df['price_min'] + df['price_max']) / 2

# Calculate unit price per m2 (triệu đồng)
df['unit_price'] = df['price_avg'] / df['area_avg']


In [ ]:
df.head(5)

In [ ]:
df.to_csv("cafeland_projects.csv", index=False, encoding="utf-8-sig")

In [ ]:
print(df)

     Mã số                            Tên dự án  \
0    13354  Căn hộ chung cư Hacinco Nguyễn Xiển   
1    13350          Căn hộ chung cư Green Tower   
2    13332   Nhà Phố - Biệt Thự HCCI Hoàng Liệt   
3    13329             Khu Đô Thị Palmy Biztown   
4    13317         Căn hộ chung cư CT3 Nghĩa Đô   
..     ...                                  ...   
369  10566            Căn hộ chung cư Làng BIDV   
370  10558                        Dolphin Plaza   
371  10535                    Richland Southern   
372  10041                           Royal City   
373  10032                        Tricon Towers   

                                         Link chi tiết          Loại dự án  \
0    https://cafeland.vn/du-an/hacinco-nguyen-xien-...     Căn hộ chung cư   
1    https://cafeland.vn/du-an/green-tower-du-an-nh...     Căn hộ chung cư   
2    https://cafeland.vn/du-an/hcci-hoang-liet-du-a...  Nhà Phố - Biệt Thự   
3    https://cafeland.vn/du-an/palmy-biztown-du-an-...          Khu Đô Thị 

In [ ]:
from google.colab import files
files.download('cafeland_projects.csv')


In [ ]:
#Call the API to get districts from longtitude and latitude
import requests

def get_district(lat, lon, api_key):
    url = f"https://mapapis.openmap.vn/v1/geocode/reverse?latlng={lat},{lon}&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK' and data['results']:
            for component in data['results'][0]['address_components']:
                if 'quận' in component['long_name']:
                    return component['long_name']
    return None

# Example usage
latitude = 21.028511
longitude = 105.804817
api_key = 'YOUR_API_KEY'
district = get_district(latitude, longitude, api_key)
print(district)  # Output: quận Ba Đình


None


In [ ]:
pip install pandas geopy


In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd

df = pd.read_csv('cafeland_projects.csv')

geolocator = Nominatim(user_agent="district_locator", timeout=10)  # Increase timeout
geocode = RateLimiter(
    geolocator.reverse,
    min_delay_seconds=2,       # Respect rate limits
    max_retries=3,
    error_wait_seconds=5,
    swallow_exceptions=True
)

def get_district(lat, lon):
    # Check if lat and lon are valid numbers before proceeding
    if pd.notna(lat) and pd.notna(lon):
        location = geocode((lat, lon), language='vi')
        if location and 'address' in location.raw:
            addr = location.raw['address']
            return addr.get('suburb') or addr.get('district') or addr.get('county')
    return None # Return None if lat or lon is invalid

df['district'] = df.apply(lambda row: get_district(row['latitude'], row['longitude']), axis=1)
df.to_csv('cafeland_with_district.csv', index=False)

In [ ]:
from google.colab import files
files.download('cafeland_with_district.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Save with correct UTF-8 encoding
try:
    df = pd.read_csv("cafeland_with_district.csv", encoding="utf-8")
except UnicodeDecodeError:
    # If that fails, try with 'utf-8-sig' or 'cp1258'
    df = pd.read_csv("cafeland_with_district.csv", encoding="utf-8-sig")  # or 'cp1258' for Windows


In [ ]:
df.to_csv("cafeland_cleaned.csv", encoding="utf-8-sig", index=False)


In [ ]:
from google.colab import files
files.download('cafeland_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#After saving I realized some rows are missing the district values
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from google.colab import files

# Upload the file 'No_district.csv' to the Colab environment
uploaded = files.upload()

# Load your file
df = pd.read_csv("No_district.csv")

# Set up geolocator
geolocator = Nominatim(user_agent="district_locator")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)

# Function to get district
def fetch_district(lat, lon):
    try:
        location = reverse((lat, lon), language="vi", addressdetails=True)
        if location and 'address' in location.raw:
            address = location.raw['address']
            return address.get('suburb') or address.get('city_district') or address.get('district') or address.get('town')
    except:
        return None

# Apply to DataFrame
df['district'] = df.apply(lambda row: fetch_district(row['latitude'], row['longitude']), axis=1)



Saving No_district.csv to No_district.csv


✅ Districts filled and saved as No_district_filled.csv


In [ ]:
# Save result
df.to_csv("No_district_filled.csv", encoding="utf-8-sig",index=False)
print("✅ Districts filled and saved as No_district_filled.csv")

✅ Districts filled and saved as No_district_filled.csv


In [ ]:
from google.colab import files
files.download('No_district_filled.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>